### Only run if corrected dataset is not available already, or to check correctness

This notebook merges the dataset with articles with the dataset cotinaing the full articles.

In [4]:
# Import necessary libraries
import pandas as pd
import os

In [ ]:
# Setting the directory
data_dir = ('\\data')

In [16]:
# Read in both summarized and full paragraphs dataframes
df = pd.read_csv(os.getcwd() + data_dir + '\\full_dataset.csv')
df_summary = pd.read_csv(os.getcwd() + data_dir + '\\articles_summary_cleaned.csv')

In [125]:
# Check the shape of the dataframes
print('unique values for each column:')
for i in df.columns:
    print(i,len(df[i].unique()))
print('shape of merged df:', df.shape)

unique values for each column:
summary 19885
date 3464
name 5813
lat 2840
lng 2750
id 19885
paragraphs 19796
publisher 276
tags 5318
shape of merged df: (329024, 9)


In [128]:
# return the first row of each group
df2 = df.groupby(['id', 'name']).first().reset_index()

df_summary.set_index(df_summary['summary'] + df_summary['location_article'], inplace=True)
# df_summary['key'] = pd.util.hash_pandas_object(df_summary['key'])

df2.set_index(df2['summary'] + df2['name'], inplace=True)
# df2['key'] = pd.util.hash_pandas_object(df2['key'])

# join df_summary and df2 on the index using df2 as master.
df_merged = pd.merge(df2, df_summary, how='inner', left_index=True, right_index=True, suffixes=('', '_remove')).reset_index()

# drop duplicate columns from the join where the values are the same
df_merged.drop([i for i in df_merged.columns if 'remove' in i],
               axis=1, inplace=True)
# drop location_article and key columns as well
df_merged.drop(['location_article', 'id', 'key'], axis=1, inplace=True)

# show the first 5 rows of the merged df
df_merged.head()

,index,name,summary,date,lat,lng,paragraphs,publisher,tags
0,The article discusses the passing of the new C...,Juba,The article discusses the passing of the new C...,2011-07-07,4.859363,31.571250,Juba — THE parliament of South Sudan has passe...,New Vision (Kampala),"['South Sudan', 'East Africa', 'Governance', '..."
1,The article discusses the military actions tak...,Abyei,The article discusses the military actions tak...,2011-07-03,9.838551,28.486396,What is the logic of Khartoum's military actio...,South Sudan News Agency,"['South Sudan', 'East Africa', 'Business', 'Co..."
2,The article discusses the signing of a Framewo...,Southern Kordofan,The article discusses the signing of a Framewo...,2011-06-30,11.036544,30.895824,Secretary-General Ban Ki-moon today welcomed t...,UN News Service,"['Sudan', 'East Africa', 'Governance', 'Extern..."
3,The article discusses the upcoming independenc...,South Sudan,The article discusses the upcoming independenc...,2011-07-04,6.876992,31.306979,AS South Sudan readies itself to become the wo...,The Moment (London),"['Sudan', 'East Africa', 'Africa', 'South Sudan']"
4,The article discusses the need for South Sudan...,Juba,The article discusses the need for South Sudan...,2011-07-02,4.859363,31.571250,Juba — The Vice President of the soon-to-be in...,Sudan Tribune,"['South Sudan', 'East Africa', 'Business', 'Ex..."


In [129]:
# save the merged df to a csv file of the name 'corrected_full_dataset.csv'
df_merged.to_csv(os.getcwd() + data_dir + '\\corrected_full_dataset.csv', index=False)